In [1]:
import cufflinks as cf
import pandas as pd
import numpy as np
from pandasql import PandaSQL
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import plot, iplot

In [2]:
cf.go_offline()

In [3]:
# Importando os arquivos
df_employers = pd.read_parquet('gold/employers.parquet')
df_cargos = pd.read_parquet('gold/cargos.parquet')
df_cep = pd.read_parquet('gold/cep.parquet')
df_base_levels = pd.read_parquet('gold/base_levels.parquet')
df_clients = pd.read_parquet('gold/clients.parquet')

## VISUALIZAÇÃO E GERAÇÃO DE INSIGHTS

<p>FUNCIONÁRIOS</p>

- Dados de Contratação e Demissão:
	- Taxa de rotatividade.
- Relacionamento com Cargos:
	- Relação entre cargo e nível.

<p> Taxa de rotatividade (%) </p>

In [4]:
# Adiciona coluna de ano
df_employers['ano_contratacao'] = df_employers['data_de_contratacao'].dt.year

# Inicializa a classe PandaSQL
pysqldf = PandaSQL()

# Calcula a taxa de rotatividade ano a ano usando pandasql
query_rotatividade_ano_a_ano = """
    SELECT 
        ano_contratacao,
        COUNT(DISTINCT id_rh) AS total_funcionarios,
        SUM(CASE WHEN data_de_demissao IS NOT NULL THEN 1 ELSE 0 END) AS funcionarios_que_deixaram,
        100 * SUM(CASE WHEN data_de_demissao IS NOT NULL THEN 1 ELSE 0 END) / COUNT(DISTINCT id_rh) AS taxa_rotatividade
    FROM df_employers
    GROUP BY ano_contratacao
    ORDER BY ano_contratacao
"""

resultado_rotatividade_ano_a_ano = pysqldf(query_rotatividade_ano_a_ano)

ano_contratacao = resultado_rotatividade_ano_a_ano['ano_contratacao']
taxa_rotatividade = resultado_rotatividade_ano_a_ano['taxa_rotatividade']
total_funcionarios = resultado_rotatividade_ano_a_ano['total_funcionarios']

fig = go.Figure()

# Adicionar linha para taxa de rotatividade
fig.add_trace(go.Scatter(x=ano_contratacao, y=taxa_rotatividade, mode='markers+lines', name='Taxa de Rotatividade'))

# Adicionar linha para total de funcionários
fig.add_trace(go.Scatter(x=ano_contratacao, y=total_funcionarios, mode='markers+lines', name='Total de Funcionários'))

# Adicionar rótulos interativos
fig.update_layout(hovermode='x unified')

fig.show()


Pode ser visualizado um crescimento consistente no quadro de funcionários ao longo do tempo, refletindo o progresso contínuo da empresa. A taxa de rotatividade (Turnover), por sua vez, tem mantido uma tendência modesta e geralmente alinhada aos movimentos do mercado. Para uma análise mais aprofundada e aprimoramento contínuo dessa métrica, é possível explorar alguns aspectos adicionais. Estratégias voltadas para a promoção da Saúde Organizacional, aumento da Satisfação dos Funcionários e Investimentos em Desenvolvimento Profissional podem ser implementadas e monitoradas de perto. Essas iniciativas têm o potencial de não apenas otimizar a taxa de rotatividade, mas também promover um ambiente de trabalho mais saudável e produtivo.

### <p>Distribuição de Funcionários por Cargo e Nível (Ativos)</p>

In [5]:
# Calcular a distribuição de funcionários ativos por cargo e nível usando pandasql
dist_func_cargo_nivel_query = """
    SELECT 
        c.area as DescricaoCargo,
        c.nivel as Nivel,
        COUNT(*) as Quantidade
    FROM df_employers e
    LEFT JOIN df_cargos c ON e.cargo = c.cargo
    WHERE e.data_de_demissao IS NULL
    GROUP BY DescricaoCargo, c.nivel
"""

resultado_dist_func_cargo_nivel = pysqldf(dist_func_cargo_nivel_query)

# Criando visualização
fig = px.bar(resultado_dist_func_cargo_nivel, x='DescricaoCargo', y='Quantidade', color='Nivel',
             labels={'Quantidade': 'Quantidade de Funcionários'},
             title='Distribuição de Funcionários por Cargo e Nível',
             height=500)

# Adicionando legenda apenas para Nível e Cargo
fig.update_layout(legend=dict(title=dict(text='Legenda')))
fig.show()

c:\repositorio\desafionst\desafionst\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Existe uma disparidade significativa na quantidade de estagiários em comparação com analistas especialmente nos cargos AD e OP. Notavelmente, existem 30 estagiários em relação a 27 analistas no cargo AD e 37 estagiários em relação a 21 analistas no cargo OP. Essa assimetria pode indicar uma concentração desproporcional de estagiários em certas áreas da empresa. Este insight sugere a possibilidade de concentrar esforços em programas de treinamento e desenvolvimento para estagiários, incentivando a progressão profissional e equilibrando a distribuição de talentos ao longo do tempo. 

### CLIENTES

<p>Contratos e Receitas:</p>
- TOP Clientes por Valor do Contrato Anual<br>
- TOP Quantidade De Serviços<br>
- TOP Regiões por Valor de Contrato<br>


<p>TOP Clientes por Valor do Contrato Anual</p>

In [6]:
# Consulta SQL para selecionar os TOP clientes pelo valor do contrato anual em ordem decrescente
query_top_clients = """
    SELECT cliente, valor_contrato_anual
    FROM df_clients
    ORDER BY valor_contrato_anual DESC
    LIMIT 3
"""

# Executar a consulta SQL
df_top_clients = pysqldf(query_top_clients)

# Criar gráfico de barras horizontais
fig = px.bar(df_top_clients, x='valor_contrato_anual', y='cliente', orientation='h',
             labels={'valor_contrato_anual': 'Valor do Contrato Anual', 'cliente': 'Cliente'},
             title='TOP Clientes por Valor do Contrato Anual',
             height=400)

# Exibir a figura
fig.show()

Pode-ser ver uma concentração de receita, pois o top 3 é formado por 3 áreas da empresa. Também pode fornecer uma visão estratégica sobre a importância dessas áreas para a empresa. A profunda compreensão desses setores oferece oportunidades estratégicas para otimização do relacionamento, expansão de serviços e aumento da fidelidade do cliente. Essa análise valiosa pode impulsionar decisões orientadas por dados, melhorando a eficiência operacional e a competitividade da empresa. Além de analisar a rentabilidade, ciclo de vida do cliente e satisfação do cliente, é crucial monitorar as tendências de mercado e realizar benchmarking competitivo. Uma abordagem proativa para análise de riscos, inovação de produtos e serviços, juntamente com a integração de dados de diversas fontes, proporciona uma visão mais abrangente e profunda para esse tipo de relacionamento.

<p>TOP Quantidade De Serviços</p>

In [7]:
# Consulta SQL para selecionar os TOP clientes pela quantidade de serviços
query_top_services = """
    SELECT cliente, quantidade_de_servicos
    FROM df_clients
    ORDER BY quantidade_de_servicos DESC
    LIMIT 5
"""

# Executar a consulta SQL
df_top_services = pysqldf(query_top_services)

# Criar gráfico de barras horizontais
fig = px.bar(df_top_services, x='quantidade_de_servicos', y='cliente',
             labels={'quantidade_de_servicos': 'Quantidade de Serviços', 'cliente': 'Cliente'},
             title='TOP Clientes por Quantidade de Serviços',
             height=400)

# Exibir a figura
fig.show()

Diferente do que foi visto anteriormente ao analisar os clientes que mais solicitavam serviços, percebemos algo diferente. Os clientes que têm muitos serviços não estão entre as cinco principais em termos de valor contratual. Isso sugere que alguns setores se especializam em serviços que não geram tanto dinheiro. Essa diferença destaca a importância de equilibrar a quantidade e o valor dos serviços para otimizar a receita total da empresa. Adaptar as estratégias para encontrar um equilíbrio eficaz entre a quantidade de serviços oferecidos e o valor gerado por esses serviços é crucial.

<p>TOP Regiões por Valor de Contrato</p>

In [8]:
# Consulta SQL para selecionar os TOP regiões por valor de contrato
query_top_clients = """
    SELECT reg.regiao, SUM(cli.valor_contrato_anual) as total_contrato_anual
    FROM df_clients AS cli
    INNER JOIN df_cep AS reg ON cli.cep = reg.cep
    GROUP BY reg.regiao
    ORDER BY total_contrato_anual DESC
    LIMIT 5;
"""

df_top_clients_reg = pysqldf(query_top_clients)

# Criar gráfico de barras agrupadas
fig = px.bar(df_top_clients_reg, x='regiao', y='total_contrato_anual', color='regiao',
             labels={'total_contrato_anual': 'Valor do Contrato Anual', 'regiao': 'Região'},
             title='TOP Regiões por Valor de Contrato',
             height=500)
             

# Exibir a figura
fig.show()

c:\repositorio\desafionst\desafionst\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



As regiões em destaque nos dá uma visão sobre quais partes do país estão contribuindo mais para o dinheiro que a empresa recebe por meio de contratos. Cada região tem suas características, e entender como elas se relacionam com os contratos ajuda a empresa a adaptar suas estratégias para atender melhor às necessidades dessas diferentes áreas geográficas. Isso destaca a importância de uma abordagem adaptativa e personalizada para otimizar o valor dos contratos em cada região.